In [1]:
# this demo requires existence of table "stage_expenses_cleaned" created in "demo_SQL_DataCleanup"
import pandas as pd
import sqlite3
import os

# create the sqlite DB
db = sqlite3.connect("MyDemo.db")

def run_query(query):
    return pd.read_sql_query(query,db)

os.getcwd()

'C:\\Users\\stevewatkins\\Downloads\\PythonForDataAnalysis'

In [2]:
# query to show us what tables exist in DB
query = """
SELECT * 
FROM sqlite_master
ORDER BY name;
"""
run_query(query)

,type,name,tbl_name,rootpage,sql
0,index,ix_stage_expenses_import_index,stage_expenses_import,4,"CREATE INDEX ""ix_stage_expenses_import_index""O..."
1,table,stage_expenses_cleaned,stage_expenses_cleaned,5,CREATE TABLE stage_expenses_cleaned(\n Tran_D...
2,table,stage_expenses_import,stage_expenses_import,2,"CREATE TABLE ""stage_expenses_import"" (\n""index..."


In [3]:
# get information about my staging table 
query_table_help = """
select * from PRAGMA_TABLE_INFO('stage_expenses_cleaned');
"""
run_query(query_table_help)

,cid,name,type,notnull,dflt_value,pk
0,0,Tran_Date,,0,None,0
1,1,Vendor_Name,TEXT,0,None,0
2,2,Vendor_State,TEXT,0,None,0
3,3,Vendor_Zip,TEXT,0,None,0
4,4,Vendor_Category,,0,None,0
5,5,Vendor_Amount,REAL,0,None,0


In [4]:
query = """
SELECT * 
FROM stage_expenses_cleaned;
"""
run_query(query)

,Tran_Date,Vendor_Name,Vendor_State,Vendor_Zip,Vendor_Category,Vendor_Amount
0,2018-01-01,THORNTONS,KY,40059,Transportation,7.71
1,2018-01-02,Google Services,CA,94043,Business Services,9.99
2,2018-01-02,THORNTONS,KY,40059,Transportation,11.70
3,2018-01-03,Amazon Marketplace,WA,98109,Merchandise & Supplies,85.00
4,2018-01-03,AMAZON WEB SERVICES,WA,98109,Business Services,9.46
5,2018-01-11,Trader Joe's,KY,40207,Merchandise & Supplies,147.72
6,2018-10-20,SEATGEEK,NY,10003,Entertainment,18.96
7,2018-10-22,SEATGEEK,NY,10003,Entertainment,21.08


In [5]:
# create key metrics by US state
query_state_KM = """
select 
 Vendor_State
,sum(Vendor_Amount) as "Sum Amount"
,avg(Vendor_Amount) as "Avg Amount"
,count(*) as "Trans Count"
from stage_expenses_cleaned
group by Vendor_State

union all

select 
 '(Total)'
,sum(Vendor_Amount) as "Sum Amount"
,avg(Vendor_Amount) as "Avg Amount"
,count(*) as "Trans Count"
from stage_expenses_cleaned

order by "Sum Amount" desc;
"""

run_query(query_state_KM)


,Vendor_State,Sum Amount,Avg Amount,Trans Count
0,(Total),311.62,38.9525,8
1,KY,167.13,55.7100,3
2,WA,94.46,47.2300,2
3,NY,40.04,20.0200,2
4,CA,9.99,9.9900,1


In [8]:
# create key metrics by Category
query_category_KM = """
select 
 Vendor_Category
,sum(Vendor_Amount) as "Sum Amount"
,avg(Vendor_Amount) as "Avg Amount"
,count(*) as "Trans Count"
from stage_expenses_cleaned
group by Vendor_Category

union all

select 
 '(Total)'
,sum(Vendor_Amount) 
,avg(Vendor_Amount) 
,count(*) 
from stage_expenses_cleaned

order by "Sum Amount" desc;
"""

run_query(query_category_KM)

,'(Total)',sum(Vendor_Amount),avg(Vendor_Amount),count(*)
0,(Total),311.62,38.9525,8


In [9]:
# create state count for map
query_st_map = """
select 
 Vendor_State
,count(*) as "Trans Count"
from stage_expenses_cleaned
group by Vendor_State
order by "Vendor_State" asc;
"""

run_query(query_st_map)

,Vendor_State,Trans Count
0,CA,1
1,KY,3
2,NY,2
3,WA,2
